## License 

Copyright 2021 Patrick Hall (jphall@gwu.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

*DISCLAIMER*: This notebook is not legal or compliance advice.

# Model Evaluation Notebook

#### Imports and inits

In [1]:
import os              # for directory and file manipulation
import numpy as np     # for basic array manipulation
import pandas as pd    # for dataframe manipulation
import datetime        # for timestamp

# for model eval
from sklearn.metrics import accuracy_score, f1_score, log_loss, mean_squared_error, roc_auc_score

# global constants 
ROUND = 3              # generally, insane precision is not needed 
SEED = 12345           # seed for better reproducibility

# set global random seed for better reproducibility
np.random.seed(SEED)

#### Set basic metadata

In [2]:
y_name = 'high_priced'
scores_dir = 'data/scores'

#### Read in score files 

In [3]:
# init score frame with known test y values
scores_frame = pd.read_csv(scores_dir + os.sep +'key.csv', index_col='Unnamed: 0')

# create random folds in reproducible way
np.random.seed(SEED)
scores_frame['fold'] = np.random.choice(5, scores_frame.shape[0])

# read in each score file in the directory as a new column 
for file in os.listdir(scores_dir):
    if file != 'key.csv' and file.endswith('.csv'):
        scores_frame[file[:-4]] = pd.read_csv(scores_dir + os.sep + file)['phat']

# sanity check 
scores_frame

,high_priced,fold,group3_randomforest,group4_mxgb,group2_mxgb,group5_ebm,group6_mxgb,ph_mxgb,group2_ebm,ph_glm,...,group8_ebmoversample,group8_gbmgrid,group5_mgbm,group8_ebm,group2_glm,group3_monotonicgbm,group4_ebm,group1_rf,group8_monogbm,ph_ebm
0,0.0,2,0.063028,0.058037,0.065526,0.079366,0.062775,0.059522,0.065526,0.142090,...,0.452653,0.109255,0.076263,0.082841,0.142090,0.086361,0.058037,0.100704,0.169457,0.082841
1,0.0,1,0.030348,0.032129,0.032689,0.027144,0.035047,0.036210,0.032689,0.081674,...,0.213467,0.021129,0.036957,0.027079,0.081674,0.033920,0.032129,0.098442,0.016476,0.027079
2,1.0,4,0.174309,0.161683,0.167186,0.182317,0.161925,0.180734,0.167186,0.125823,...,0.672604,0.181202,0.188531,0.190718,0.125823,0.183323,0.161683,0.119991,0.178777,0.190718
3,0.0,1,0.019114,0.023556,0.033490,0.029269,0.026129,0.027677,0.033490,0.006973,...,0.203551,0.015636,0.022215,0.031069,0.006973,0.030934,0.023556,0.065078,0.009344,0.031069
4,1.0,2,0.207948,0.180101,0.173838,0.202628,0.176872,0.177813,0.173838,0.130426,...,0.689614,0.213761,0.177027,0.210361,0.130426,0.178491,0.180101,0.117926,0.179083,0.210361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19826,0.0,3,0.245902,0.253574,0.266065,0.225395,0.250139,0.274767,0.266065,0.160032,...,0.735663,0.243438,0.230648,0.231624,0.160032,0.255826,0.253574,0.124180,0.247609,0.231624
19827,0.0,1,0.177097,0.173608,0.184479,0.252359,0.181217,0.182039,0.184479,0.123836,...,0.741521,0.249503,0.184512,0.254823,0.123836,0.176984,0.173608,0.105582,0.234628,0.254823
19828,1.0,3,0.174827,0.229630,0.217184,0.226476,0.221794,0.212740,0.217184,0.169604,...,0.723375,0.182491,0.219277,0.220400,0.169604,0.236894,0.229630,0.127146,0.192819,0.220400
19829,0.0,1,0.035276,0.001175,0.001412,0.001222,0.001602,0.001323,0.001412,0.002538,...,0.005516,0.002802,0.010324,0.000993,0.002538,0.001113,0.001175,0.056343,0.003607,0.000993


#### Utility function for max. accuracy

In [4]:
def max_acc(y, phat, res=0.01): 

    """ Utility function for finding max. accuracy at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. accuracy, default 0.01.
        :return: Max. accuracy for model scores.
    
    """
    
    # init frame to store acc at different cutoffs
    acc_frame = pd.DataFrame(columns=['cut', 'acc'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find accuracy at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        acc = accuracy_score(temp_df.iloc[:, 0], temp_df['decision'])
        acc_frame = acc_frame.append({'cut': cut,
                                      'acc': acc},
                                     ignore_index=True)

    # find max accurcay across all cutoffs
    max_acc = acc_frame['acc'].max()
    
    # house keeping
    del acc_frame, temp_df
    
    return max_acc

####  Utility function for max. F1

In [5]:
def max_f1(y, phat, res=0.01): 
    
    """ Utility function for finding max. F1 at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. F1, default 0.01.
        :return: Max. F1 for model scores.
    
    """
    
    # init frame to store f1 at different cutoffs
    f1_frame = pd.DataFrame(columns=['cut', 'f1'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find f1 at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        f1 = f1_score(temp_df.iloc[:, 0], temp_df['decision'])
        f1_frame = f1_frame.append({'cut': cut,
                                    'f1': f1},
                                    ignore_index=True)
        
    # find max f1 across all cutoffs
    max_f1 = f1_frame['f1'].max()
    
     # house keeping
    del f1_frame, temp_df
    
    return max_f1

#### Rank all submitted scores 

In [6]:
eval_frame = pd.DataFrame() # init frame to hold score ranking
metric_list = ['acc', 'auc', 'f1', 'logloss', 'mse'] # metric to use for evaluation

# create eval frame row-by-row
for fold in sorted(scores_frame['fold'].unique()): # loop through folds 
    for metric_name in metric_list: # loop through metrics
        
        # init row dict to hold each rows values
        row_dict = {'fold': fold,
                    'metric': metric_name}
        
        # cache known y values for fold
        fold_y = scores_frame.loc[scores_frame['fold'] == fold, y_name]
        
        for col_name in scores_frame.columns[2:]:
            
            # cache fold scores
            fold_scores = scores_frame.loc[scores_frame['fold'] == fold, col_name]
            
            # calculate evaluation metric for fold
            # with reasonable precision 
            
            if metric_name == 'acc':
                row_dict[col_name] = np.round(max_acc(fold_y, fold_scores), ROUND)
                
            if metric_name == 'auc':
                row_dict[col_name] = np.round(roc_auc_score(fold_y, fold_scores), ROUND)
                
            if metric_name == 'f1':
                row_dict[col_name] = np.round(max_f1(fold_y, fold_scores), ROUND) 
                
            if metric_name == 'logloss':
                row_dict[col_name] = np.round(log_loss(fold_y, fold_scores), ROUND)
                
            if metric_name == 'mse':
                row_dict[col_name] = np.round(mean_squared_error(fold_y, fold_scores), ROUND)
        
        # append row values to eval_frame
        eval_frame = eval_frame.append(row_dict, ignore_index=True)

# init a temporary frame to hold rank information
rank_names = [name + '_rank' for name in eval_frame.columns if name not in ['fold', 'metric']]
rank_frame = pd.DataFrame(columns=rank_names)        

# set columns to necessary order
eval_frame = eval_frame[['fold', 'metric'] + [name for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]]

# determine score ranks row-by-row
for i in range(0, eval_frame.shape[0]):
        
        # get ranks for row based on metric
        metric_name = eval_frame.loc[i, 'metric']
        if metric_name in ['logloss', 'mse']:
            ranks = eval_frame.iloc[i, 2:].rank().values
        else:
            ranks = eval_frame.iloc[i, 2:].rank(ascending=False).values
        
        # create single-row frame and append to rank_frame
        row_frame = pd.DataFrame(ranks.reshape(1, ranks.shape[0]), columns=rank_names)
        rank_frame = rank_frame.append(row_frame, ignore_index=True)
        
        # house keeping
        del row_frame

# merge ranks onto eval_frame
eval_frame = pd.concat([eval_frame, rank_frame], axis=1)

# house keeping
del rank_frame
        
eval_frame

,fold,metric,Group7_EBM,Group7_XGBoost,group1_ebm,group1_glm,group1_mxgb,group1_rf,group2_ebm,group2_glm,...,group5_mgbm_rank,group6_ebm_rank,group6_mxgb_rank,group8_ebm_rank,group8_ebmoversample_rank,group8_gbmgrid_rank,group8_monogbm_rank,ph_ebm_rank,ph_glm_rank,ph_mxgb_rank
0,0.0,acc,0.901,0.901,0.901,0.900,0.901,0.900,0.901,0.900,...,3.0,12.5,3.0,12.5,24.0,24.0,12.5,12.5,24.0,3.0
1,0.0,auc,0.813,0.813,0.839,0.775,0.814,0.803,0.815,0.775,...,9.0,7.5,17.0,2.5,2.5,5.0,7.5,2.5,26.5,17.0
2,0.0,f1,0.375,0.375,0.408,0.335,0.377,0.368,0.379,0.335,...,9.0,7.0,13.5,2.0,6.0,8.0,5.0,2.0,25.5,11.0
3,0.0,logloss,0.263,0.263,0.251,0.291,0.263,0.305,0.263,0.291,...,8.0,7.0,15.0,2.5,28.0,2.5,5.5,2.5,23.5,15.0
4,0.0,mse,0.078,0.078,0.077,0.084,0.079,0.086,0.078,0.084,...,4.5,4.5,18.5,4.5,28.0,4.5,4.5,4.5,23.5,12.0
5,1.0,acc,0.906,0.906,0.906,0.906,0.906,0.906,0.906,0.906,...,14.5,14.5,14.5,14.5,14.5,14.5,14.5,14.5,14.5,14.5
6,1.0,auc,0.795,0.795,0.828,0.757,0.791,0.792,0.793,0.757,...,9.0,8.0,21.5,5.0,2.0,2.0,7.0,5.0,26.0,17.5
7,1.0,f1,0.341,0.341,0.369,0.302,0.339,0.336,0.339,0.302,...,8.5,7.0,13.0,4.5,1.5,1.5,8.5,4.5,26.0,20.0
8,1.0,logloss,0.262,0.262,0.246,0.281,0.263,0.294,0.263,0.281,...,8.0,6.5,17.5,3.0,28.0,3.0,6.5,3.0,23.5,17.5
9,1.0,mse,0.078,0.078,0.074,0.080,0.078,0.082,0.078,0.080,...,6.5,6.5,15.0,2.5,28.0,6.5,6.5,2.5,23.5,15.0


#### Save `eval_frame` as CSV

In [7]:
eval_frame.to_csv('model_eval_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                  index=False)

#### Display simple ranked score list 

In [8]:
eval_frame[[name for name in eval_frame.columns if name.endswith('rank')]].mean().sort_values()

group1_ebm_rank               5.00
ph_ebm_rank                   5.00
group8_ebm_rank               5.00
group8_gbmgrid_rank           5.68
group5_ebm_rank               6.22
group6_ebm_rank               6.66
group8_monogbm_rank           6.84
group5_mgbm_rank              7.98
group3_decisiontree_rank     14.02
Group7_EBM_rank              14.88
Group7_XGBoost_rank          14.88
group1_mxgb_rank             15.02
group2_ebm_rank              15.14
group2_mxgb_rank             15.14
group8_ebmoversample_rank    15.20
group6_mxgb_rank             15.40
group3_randomforest_rank     15.44
ph_mxgb_rank                 15.56
group3_monotonicgbm_rank     16.02
group3_ebm_rank              16.02
group4_ebm_rank              16.36
group4_mxgb_rank             16.36
group1_rf_rank               23.20
ph_glm_rank                  23.68
group3_elasticnet_rank       23.68
group2_glm_rank              23.68
group1_glm_rank              23.68
group4_glm_rank              24.26
dtype: float64